# SSD 300 model kvantizacia

Postup ako pri ResNet

Model je predtrénovaný na COCO datasete.
Model z:
https://pytorch.org/hub/nvidia_deeplearningexamples_ssd/
https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD

Definícia modelu prebratá z:
https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Detection/SSD/src/model.py
a upravená.

Model upravíme pre potreby kvantizácie, úprava modelu podľa tutorialu v Distiller Framework.

## Loading pre-trained model

In [ ]:
import torch

In [ ]:
torch.cuda.empty_cache()
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

In [ ]:
ssd_model


In [ ]:
#loading pre-processing utils for model

utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Chrbtica tohto SSD modelu je ResNet, môžeme teda priamo použiť ResNet implementovaný v rámci Distiller

### Base precision

Potrebujeme si uložiť váhy tohto modelu, následne vyrobiť impl pre Distiler kvantizaciu a dayť váhy tomuto novému modelu.

In [ ]:
import os
chckpt_name = "./ssd_model_ORIG.pth"
if os.path.isfile(chckpt_name):
    torch.save(ssd_model,chckpt_name)


In [ ]:
#ssd_model.to('cuda')
#ssd_model.eval()


In [ ]:
import torch.utils
import torchvision


COCO_PATH = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017"
COCO_ANN = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/annotations/instances_val2017.json"
# 
# dataset = torchvision.datasets.CocoDetection(root= COCO_PATH,annFile= COCO_ANN, transform=utils.prepare_tensor)
# 
# #musime pouzit torch dataloader, lebo distiler nema coco impl
# subset_size = 1.0
# val_split = 0.8
# 
# batch_size_gpu = 32
# workers_gpu = 0
# 
# dataloader = torch.utils.data.DataLoader(dataset, batch_size= batch_size_gpu, num_workers=workers_gpu)



In [ ]:
def get_images_list(dir):
    # f= []
    return [ os.path.join(dir,f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir,f))]


In [ ]:
import os
images_list = get_images_list(COCO_PATH)
len(images_list)

In [ ]:
def prepare(images_list, prec):
    inputs = [utils.prepare_input(uri) for uri in images_list]
    tensor = utils.prepare_tensor(inputs, precision == prec)
    return inputs,tensor

In [ ]:
#torch.cuda.empty_cache()
#part = images_list[:25]
#%time inputs, tensor = prepare(part, 'fp16')



In [ ]:
# results_per_input = utils.decode_results(detections_batch)
# best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [ ]:
# classes_to_labels = utils.get_coco_object_dictionary()

In [ ]:
# from matplotlib import pyplot as plt
# import matplotlib.patches as patches
# 
# for image_idx in range(len(best_results_per_input)):
#     fig, ax = plt.subplots(1)
#     # Show original, denormalized image...
#     image = inputs[image_idx] / 2 + 0.5
#     ax.imshow(image)
#     # ...with detections
#     bboxes, classes, confidences = best_results_per_input[image_idx]
#     for idx in range(len(bboxes)):
#         left, bot, right, top = bboxes[idx]
#         x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
#         rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)
#         ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
# plt.show()



In [ ]:
print('ahoj')

In [ ]:
from src.model import SSD300 , ResNet

ssd_dist = SSD300()

In [ ]:
ssd_dist

In [ ]:
# ssd_dist2 = SSD300(backbone=ResNet(backbone='resnet18'))

In [ ]:
part = images_list[:10]
inputs, tensor = prepare(part, 'fp16')

In [ ]:
part

In [ ]:
# ssd_dist.to('cuda')
# ssd_dist.eval()

In [ ]:
# eval(inputs, tensor, utils, ssd_dist)


In [ ]:
# torch.cuda.empty_cache()
ssd_dist.to('cpu')

ssd_model.to('cuda')
ssd_model.eval()

In [ ]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

In [ ]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [ ]:
classes_to_labels = utils.get_coco_object_dictionary()

In [ ]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(classes_to_labels, fp)

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches

for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized image...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ...with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], 
                                          confidences[idx]*100), bbox=dict(facecolor='white', 
                                                                           alpha=0.5))
plt.show()